# Custom Advanced Neural Network for MedMNIST Leaderboard Submission Using Kaggle Input Data

This notebook loads MedMNIST datasets directly from the Kaggle input folder (using provided NPZ files), converts the images into TensorFlow datasets (with data augmentation and normalization), and builds a custom CNN with advanced techniques (including residual blocks, batch normalization, dropout, and a cosine-decay learning rate scheduler).  
A separate model is trained for each task (excluding ChestMNIST) for up to **100 epochs** with early stopping, and the final predictions are combined into a submission CSV file with columns: `id`, `id_image_in_task`, `task_name`, and `label`.

In [1]:
import tensorflow as tf

# Check for available GPUs and enable dynamic memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPUs found and configured:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Please set your runtime to GPU in the Kaggle Notebook settings.")

GPUs found and configured: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
!pip install -q medmnist

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import f1_score
import medmnist
from medmnist import INFO
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# MedMNIST images are 28x28. We expand them to 3 channels.
TARGET_SIZE = (28, 28)  
BATCH_SIZE = 32

In [4]:
def load_npz_data(npz_path):
    """
    Load train, val, and test arrays from a given NPZ file.
    Expected keys: 'train_images', 'train_labels', 'val_images', 'val_labels', 'test_images', 'test_labels'
    """
    data = np.load(npz_path)
    train_images = data['train_images']
    train_labels = data['train_labels']
    val_images = data['val_images']
    val_labels = data['val_labels']
    test_images = data['test_images']
    test_labels = data['test_labels']
    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

def create_tf_dataset_from_numpy(images, labels, batch_size=BATCH_SIZE, augment=False):
    """
    Convert numpy arrays to a tf.data.Dataset:
      - Expand grayscale images (28x28) to (28,28,1) then tile to (28,28,3).
      - Normalize pixel values to [0,1].
      - Optionally apply data augmentation.
    """
    if images.ndim == 3:
        images = np.expand_dims(images, axis=-1)  # (N,28,28,1)
    if images.shape[-1] == 1:
        images = np.tile(images, (1, 1, 1, 3))       # (N,28,28,3)
    
    images = images.astype(np.float32) / 255.0
    
    def _process(image, label):
        # No resizing needed (TARGET_SIZE equals original size)
        if augment:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)
        return image, label

    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ds = ds.map(_process, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(images))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [5]:
# Define the list of tasks (excluding ChestMNIST)
task_names = [
    "pathmnist",
    "dermamnist",   # In INFO, the python_class is "DermaMNIST"
    "octmnist",
    "pneumoniamnist",
    "retinamnist",
    "breastmnist",
    "bloodmnist",
    "tissuemnist",
    "organamnist",
    "organcmnist",
    "organsmnist"
]

# Use the Kaggle input folder (no downloading occurs)
base_path = Path("/kaggle/input/tensor-reloaded-multi-task-med-mnist/data")
task_to_npz = {task: base_path / f"{task}.npz" for task in task_names}

# Load datasets for each task from the NPZ files
train_datasets = {}
val_datasets = {}
test_datasets = {}

for task in task_names:
    (train_imgs, train_lbls), (val_imgs, val_lbls), (test_imgs, test_lbls) = load_npz_data(task_to_npz[task])
    train_datasets[task] = (train_imgs, train_lbls)
    val_datasets[task] = (val_imgs, val_lbls)
    test_datasets[task] = (test_imgs, test_lbls)
    print(f"{task}: {len(train_imgs)} train, {len(val_imgs)} val, {len(test_imgs)} test samples")

pathmnist: 89996 train, 10004 val, 7180 test samples
dermamnist: 7007 train, 1003 val, 2005 test samples
octmnist: 97477 train, 10832 val, 1000 test samples
pneumoniamnist: 4708 train, 524 val, 624 test samples
retinamnist: 1080 train, 120 val, 400 test samples
breastmnist: 546 train, 78 val, 156 test samples
bloodmnist: 11959 train, 1712 val, 3421 test samples
tissuemnist: 165466 train, 23640 val, 47280 test samples
organamnist: 34581 train, 6491 val, 17778 test samples
organcmnist: 13000 train, 2392 val, 8268 test samples
organsmnist: 13940 train, 2452 val, 8829 test samples


In [6]:
train_datasets_tf = {}
val_datasets_tf = {}
test_datasets_tf = {}

for task in task_names:
    train_imgs, train_lbls = train_datasets[task]
    val_imgs, val_lbls = val_datasets[task]
    test_imgs, test_lbls = test_datasets[task]
    
    train_datasets_tf[task] = create_tf_dataset_from_numpy(train_imgs, train_lbls, batch_size=BATCH_SIZE, augment=True)
    val_datasets_tf[task]   = create_tf_dataset_from_numpy(val_imgs, val_lbls, batch_size=BATCH_SIZE, augment=False)
    test_datasets_tf[task]  = create_tf_dataset_from_numpy(test_imgs, test_lbls, batch_size=BATCH_SIZE, augment=False)
    
    count = sum(1 for _ in test_datasets_tf[task])
    print(f"{task}: Test batches: {count}")

pathmnist: Test batches: 225
dermamnist: Test batches: 63
octmnist: Test batches: 32
pneumoniamnist: Test batches: 20
retinamnist: Test batches: 13
breastmnist: Test batches: 5
bloodmnist: Test batches: 107
tissuemnist: Test batches: 1478
organamnist: Test batches: 556
organcmnist: Test batches: 259
organsmnist: Test batches: 276


In [7]:
def residual_block(x, filters, kernel_size=(3,3), strides=(1,1), dropout_rate=0.1):
    shortcut = x
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, strides=(1,1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if dropout_rate > 0:
        x = layers.Dropout(dropout_rate)(x)
    # If shortcut shape doesn't match, project it
    if shortcut.shape[-1] != filters or strides != (1,1):
        shortcut = layers.Conv2D(filters, (1,1), strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

def build_custom_model(num_classes, input_shape=(28,28,3)):
    inputs = keras.Input(shape=input_shape)
    
    # Initial Conv block
    x = layers.Conv2D(32, (3,3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2,2))(x)  # 14x14
    
    # Residual blocks
    x = residual_block(x, 64, dropout_rate=0.2)  # 14x14
    x = layers.MaxPooling2D((2,2))(x)  # 7x7
    x = residual_block(x, 128, dropout_rate=0.3)  # 7x7
    
    # Global pooling and dense layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# (Optional test)
num_classes = len(INFO["pathmnist"]['label'])
custom_model = build_custom_model(num_classes, input_shape=(28,28,3))
custom_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 28, 28, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 28, 28, 3)      │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 28, 28, 32)     │            896 │ cast[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 28, 28, 32)     │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 28, 28, 32)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 14, 14, 32)     │              0 │ re_lu[0][0]            │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 14, 14, 64)     │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 14, 14, 64)     │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 14, 14, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 14, 14, 64)     │         36,928 │ re_lu_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 14, 14, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 14, 14, 64)     │          2,112 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 14, 14, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 14, 14, 64)     │            256 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 14, 14, 64)     │              0 │ dropout[0][0],         │
│                           │                        │                │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_2 (ReLU)            │ (None, 14, 14, 64)     │              0 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1      

 Total params: 325,961 (1.24 MB)

 Trainable params: 324,745 (1.24 MB)

 Non-trainable params: 1,216 (4.75 KB)

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

models = {}
histories = {}

for task in task_names:
    num_classes = len(INFO[task]['label'])
    print(f"\nTraining custom advanced model for {task} with {num_classes} classes...")
    model_task = build_custom_model(num_classes, input_shape=(28,28,3))
    history = model_task.fit(
        train_datasets_tf[task],
        validation_data=val_datasets_tf[task],
        epochs=100,
        callbacks=[early_stopping],
        verbose=1
    )
    models[task] = model_task
    histories[task] = history


Training custom advanced model for pathmnist with 9 classes...
Epoch 1/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 39s 7ms/step - accuracy: 0.7004 - loss: 0.8289 - val_accuracy: 0.2269 - val_loss: 5.2879
Epoch 2/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.8767 - loss: 0.3546 - val_accuracy: 0.3731 - val_loss: 3.8609
Epoch 3/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9032 - loss: 0.2796 - val_accuracy: 0.3209 - val_loss: 3.4392
Epoch 4/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9223 - loss: 0.2276 - val_accuracy: 0.4543 - val_loss: 4.0107
Epoch 5/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9354 - loss: 0.1911 - val_accuracy: 0.4039 - val_loss: 4.4638
Epoch 6/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9411 - loss: 0.1710 - val_accuracy: 0.6852 - val_loss: 1.8380
Epoch 7/100
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9498 - loss: 0.1507 - val_accuracy: 0.5512 - val_loss: 2.6748
Epoch 8/100


In [9]:
submission_rows = []
global_id = 0

for task in task_names:
    model_task = models[task]
    preds_list = []
    for images, _ in test_datasets_tf[task]:
        preds = model_task.predict(images)
        preds = np.argmax(preds, axis=1)
        preds_list.append(preds)
    preds_all = np.concatenate(preds_list)
    for idx, pred in enumerate(preds_all):
        submission_rows.append([global_id, idx, task, int(pred)])
        global_id += 1

submission_df = pd.DataFrame(submission_rows, columns=["id", "id_image_in_task", "task_name", "label"])
print("Total submission rows:", len(submission_df))
submission_df.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 673ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━